# Step 1: Best Contour Extraction/Prediction

In [ ]:
orig_dir = 'CV-Assignment-Dataset'   #Test Images Directory
dest_dir = 'plot'                    #images stored with ROI boxes

In [112]:
from image_process import (
    threshold,
    morphology,
    find_contours,
    extra_contour_elimination,
    crop_contour,
    req_contour
)
import csv
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [113]:
from keras.models import load_model
model = load_model('model.hdf5')

In [133]:
def plot(filename,file,target):
    img = cv2.imread(filename)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    clone = img.copy()
    thresh = threshold(img)
    img, lines = find_contours(thresh)
    final_lines = extra_contour_elimination(lines)
    req = []
    for i,line in enumerate(final_lines):
        L = crop_contour(line,thresh)
        img = cv2.resize(L,(28,28))
        data = img.reshape(1,28,28,1)
        model_out = model.predict([data])

        if np.argmax(model_out)==1:
            str_label = '0'
        else:
            str_label = '1'

        if str_label=='1':
            print(file)
            req.append(i)
            target.append([file,L,line])
            name = dest_dir+'/'+file
#             print(name)
            

    for index in req:
        rect = cv2.minAreaRect(final_lines[index])
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(clone,[box],0,(0,255,0),2)
        cv2.imwrite(name,clone)
    return target

In [134]:
target = []
for file in os.listdir(orig_dir):
    path = os.path.join(orig_dir,file)
    target = plot(path,file,target)

DSC08876.JPG
DJI_0422.JPG
DJI_0422.JPG
DJI_0083.JPG
DJI_0083.JPG
DSC01453.JPG
M1_F1.3_0405.JPG
M1_F1.3_0405.JPG
DJI_0617.JPG
DJI_0036.JPG
DJI_0616.JPG
DJI_0086.JPG
DJI_0086.JPG
DJI_0086.JPG
DJI_0086.JPG


# Step 2: Point of L extraction

Extraction using Center of mass, since the required coordinates lie near COM of L

In [138]:
# def get_coordinates(line)    
def get_com(contours):
#     filename = 'data/train/1/230.jpeg'
#     image = cv2.imread(filename)
#     imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     gray = imgray.copy()
#     _, thresh = cv2.threshold(imgray, 220, 255, 0)
#     _, contours, hierarchy = cv2.findContours(thresh, cv2.cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # cnts = cv2.drawContours(image, contours[0], -1, (0, 255, 0), 1)

    kpCnt = len(contours)

    x = 0
    y = 0

    for kp in contours:
        x = x+kp[0][0]
        y = y+kp[0][1]

    #     cv2.circle(image, (np.uint8(np.ceil(x/kpCnt)), np.uint8(np.ceil(y/kpCnt))), 1, (0, 255, 255),1 )

    com = np.array([x/kpCnt,y/kpCnt])
    return com

    #     print((x/kpCnt),(y/kpCnt))
    #     plt.imshow(cnts)

In [149]:
def get_gcp(gray,com):
    dst = cv2.cornerHarris(gray,2,3,0.04)
    dst = cv2.dilate(dst,None)
    ret, dst = cv2.threshold(dst,0.001*dst.max(),255,0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)


    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray,np.float32(centroids),(3,3),(-1,-1),criteria)

    dist_min = 1000000
    for corner in corners:
        euc_dist = np.linalg.norm(corner - com)
        if euc_dist<dist_min:
            dist_min = euc_dist
            gcp = corner
    return gcp
# Now draw them
# res = np.hstack((centroids,corners))
# res = np.int0(res)
# image[res[:,1],res[:,0]]=[0,0,255]
# image[res[:,3],res[:,2]] = [0,255,0]

# cv2.imwrite('subpixel5.png',image)
# plt.imshow(image)
# print(corners)
# print(gray[10,9])

In [150]:
com = get_com(target[1][2])
gcp = get_gcp(target[1][1],com)
print(com,gcp)

[2600.97142857 2253.78571429] [34.864983 18.015331]
